In [1]:
from matplotlib import pyplot as plt
from graph import create_random_graph , to_useful, assign_states
import networkx as nx
from model import create_model
import tensorflow as tf
from mutate import mutate_dag
from crossover import crossover
from Individual import Individual
from Generation import Generation

2023-12-15 06:21:34.962272: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-15 06:21:35.006932: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-15 06:21:35.006985: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-15 06:21:35.008306: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-15 06:21:35.016174: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-15 06:21:35.017472: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
import tensorflow_datasets as tfds
import tensorflow as tf

# Load CIFAR-100 dataset
cifar100_dataset, cifar100_info = tfds.load(name='cifar100', with_info=True, as_supervised=True)

# Split the dataset into train and test
train_data = cifar100_dataset['train']
test_data = cifar100_dataset['test']

# Define the preprocessing function
def preprocess(image, label):
    image = tf.cast(image, tf.float32)
    image = image / 255.0  # Scale pixel values to [0, 1]
    label = tf.cast(label, tf.int32)
    label = tf.one_hot(label, 100)
    return image, label

# Apply preprocessing to train and test data
train_data = train_data.map(preprocess)
test_data = test_data.map(preprocess)

# Shuffle train data and split into train and validation sets
train_data = train_data.shuffle(50000)
train_data = train_data.batch(128)
valid_data = train_data.take(5000)
train_data = train_data.skip(5000).prefetch(1).cache()
test_data = test_data.batch(128).prefetch(1).cache()
test_data = test_data.cache()
# Print dataset information
print(cifar100_info)


/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


(1, 128, 128, 1)
(1, 6)


In [3]:
model  =  tf.keras.models.load_model('generation_2/3.8900124160983323')
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
loss = tf.keras.losses.CategoricalCrossentropy()
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=0.0)
model.compile(optimizer=optimizer, loss=loss, metrics=[tf.keras.metrics.CategoricalAccuracy(),tf.keras.metrics.AUC(),tf.keras.metrics.Accuracy()]
)
model.summary()
model.fit(train_ds, epochs=10000, validation_data=test_ds,batch_size=64,verbose=1
          ,callbacks=[reduce_lr])

Model: "model_16"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_layer (InputLayer)    [(None, 128, 128, 1)]        0         []                            
                                                                                                  
 1-1.1-0conv2d (Conv2DPadLa  (None, 125, 125, 16)         272       ['input_layer[0][0]']         
 yer)                                                                                             
                                                                                                  
 1-4_flatten (Flatten)       (None, 250000)               0         ['1-1.1-0conv2d[0][0]']       
                                                                                                  
 2-0_flatten (Flatten)       (None, 250000)               0         ['1-4_flatten[0][0]']  

KeyboardInterrupt: 